In [15]:
#!pip install -U spacy
#!python -m spacy download es_core_news_sm
##!python -m spacy validate
##!python -m spacy link es_core_news_sm en --force
nltk.download('stopwords')
!pip install gensim 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached https://files.pythonhosted.org/packages/3a/bc/1415be59292a23ff123298b4b46ec4be80b3bfe72c8d188b58ab2653dee4/gensim-3.8.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/16/8a/1fc3dba0c4923c2a76e1ff0d52b305c44606da63f718d14d3231e21c51b0/s3transfer-0.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/83/94/7179c3832a6d45b266ddb2aac329e101367fbdb11f425f13771d27f225bb/jmespath-0.9.4-py2.py3-none-any.whl
  Running setup.py install for gensim: started
    Running setup.py install for gensim: finished with status 'done'


In [14]:
#librerias generales
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
from os import listdir
import statsmodels.tsa.stattools as stt
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import re
import warnings
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

#librerias para los modelos de series de tiempo
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, DynamicVAR

#Libresrias de pre-procesamiento de texto
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
spanish_stopwords = nltk.corpus.stopwords.words('spanish')
spanish_stopwords.extend(['Junta','Directiva','semestre','bien','entornar','aunque','respictavemente', 'probable',
                         'menos', 'me', '3', 'millón', 'punto', 'si', 'futuro', 'mostrar', 'septiembre', 'enero','1', 'b'
                         'febrero', 'marzo', 'abril', 'mayo', 'junio','julio','agosto', 'octubre', 'noviembre','diciembre'
                          
                         'anterior', 'frente', 'largar', 'tiempo', 'partir', 'cercar', 'señalar', 'mes', 'registrar'
                         'mesar', 'atrás', 'estimar', 'proyectar','miembro', 'europa', 'pb', 'finalmente', 'observar'
                         'registrar', 'segundar', 'sobrar', 'rango', 'ingresar' ])
import scipy as sc
##nlp = sc.load('es_core_news_sm')
import es_core_news_sm
nlp = es_core_news_sm.load()

#Librerias para modelo LDA
import gensim
from gensim import corpora, models

ModuleNotFoundError: No module named 'es_core_news_sm'

## Cargar series de tiempo en dataframe

In [ ]:
data = pd.read_excel(r'C:\Users\usuario\Google Drive\Varios Ivan\1 Maestria\Universidad de los Andes\ProyectoGrado\Desarrollo\dataIndicadores.xlsx', sheet_name = 'Var_Men', encoding='utf-8')

data.Fecha = pd.to_datetime(data.Fecha)
data = data.set_index('Fecha')
data = data.dropna()

cycle, trend = sm.tsa.filters.hpfilter(data.IPC, 1600)
data['IPC'] = cycle

cycle, trend = sm.tsa.filters.hpfilter(data.LIBOR, 1600)
data['LIBOR'] = cycle

for i in data.columns.tolist():
    result = stt.adfuller(data[str(i)])
    pvalue = float(result[1])
    isStationary = pvalue <= 0.05
    print(i)
    print("P-VALUE TASA: ", pvalue, isStationary)
    print('*'*40)
    print(' ')

data_series = data
data_series.tail()

## Cargar documentos en dataframe

In [ ]:
#Para listar los archivos de directorios
mypath = "C:/Users/usuario/Google Drive/Varios Ivan/1 Maestria/Universidad de los Andes/ProyectoGrado/Desarrollo/Comunicados/"
files = listdir(mypath)

#Cargar textos en un vector
docVector = []
for i in range(len(files)):
    file = open(mypath + files[i],"r", encoding="utf8") 
    #print(files[i])
    docVector.append({'text': file.read(), 'Date': files[i][:7]})

#cargar textos en un dataframe
docDataF = pd.DataFrame(docVector)
docDataF.tail()

### Función de limpieza y preprocesamiento de textos

In [ ]:
def preprocess(text, remove_stop_words=True):
    wordnet_lemmatizer = WordNetLemmatizer()
    stemmer = SnowballStemmer('spanish')
    document = text
    
    # Remove all the special characters
    document = re.sub(r'\W', ' ', document)

    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)

    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    #Removing punctuation
    document = re.sub(r'[^\w\s]', '', document)
    
    #Removing numbers
    document = ''.join(i for i in document if not i.isdigit())

    #LowerCase    
    document = document.lower()
    
    #Split document word a word
    #words_document = document.split()
    
    words_document = []    
    doc = nlp(document)
    for token in doc: words_document.append(token.lemma_)

    #Remove stop words
    if remove_stop_words:
        words_document = [word for word in words_document if word not in spanish_stopwords]
    
    #Lemmatisation
    #words_document = [wordnet_lemmatizer.lemmatize(word) for word in words_document]
    #words_document = [wordnet_lemmatizer.lemmatize(word, pos='v') for word in words_document]
    
    #stimming
    #words_document = [stemmer.stem(word) for word in words_document] 
           
    #return ' '.join(words_document)
    return words_document

### Función de vectorización

In [ ]:
def vectorizarTextos (documents_):
    dictionary = gensim.corpora.Dictionary(documents_)
    #Filter out tokens that appear in too much documents
    #less than 15 documents (absolute number) or
    #more than 0.5 documents (fraction of total corpus size, not absolute number).
    #after the above two steps, keep only the first 100000 most frequent tokens.
    dictionary.filter_extremes(no_below=20, no_above=0.9, keep_n=10000)
    #print(dictionary.token2id)
    #len(dictionary)
    bow_corpus = [dictionary.doc2bow(doc) for doc in documents_]
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
    
    return dictionary, corpus_tfidf

#### ¨prueba paralelizar
def fitLDA_ModelParallel(corpus_tfidf_ 
                         ,dictionary_ 
                         ,alpha_
                         ,rangEta = range(1, 5)
                         ,rangTopics = range(4, 10, 1)
                         ,resNumTopPala = 20):
    
    resultMatrixParalell_ = []
    for eta_ in rangEta:
        for num_topics_ in rangTopics:
            print(alpha_, eta_, num_topics_ )
            #Ajustar el modelo con la configuración que corresponda en el for
            lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf_, 
                                                         id2word = dictionary_, 
                                                         alpha = alpha_, 
                                                         eta = eta_, 
                                                         num_topics = num_topics_, 
                                                         workers=3,
                                                         random_state=42)

            #Obtener la distribución de los documentos en los tópicos
            topics = lda_model_tfidf.get_document_topics(corpus_tfidf_, 
                                                         minimum_probability=None, 
                                                         minimum_phi_value=None, 
                                                         per_word_topics=False)

            #Guarda el top n paralabra en el resultado, según parámetro resNumTopPala
            top_Palabras = []
            for i in range(0,num_topics_):
                top_Palabras.extend( np.append(np.full((resNumTopPala,1), i+1), lda_model_tfidf.show_topic(i, topn=resNumTopPala), axis=1 ) )

            resultMatrixParalell_.append([alpha_, eta_, num_topics_, topics, top_Palabras])
    
    
    return resultMatrixParalell_



def fitLDA_Model(corpus_tfidf_ 
                 ,dictionary_ 
                 ,rangAlpha = np.arange(0.1, 1, 0.1)
                 ,rangEta = range(1, 5)
                 ,rangTopics = range(4, 10, 1)
                 ,resNumTopPala = 20):
    
    resultMatrix_ = Parallel(n_jobs=-1, verbose=1)(delayed(fitLDA_ModelParallel)(corpus_tfidf_, dictionary_, alpha) for alpha in rangAlpha)
    return resultMatrix_




### Función para hallar los parámetros del modelo LDA

In [ ]:
## Función lineal - arriba está la función paralelizada
# def fitLDA_Model(corpus_tfidf_ 
#                  ,dictionary_ 
#                  ,rangAlpha = np.arange(0.4, 1.6, 0.2)
#                  ,rangEta = np.arange(0.4, 1.6, 0.2)
#                  ,rangTopics = range(4, 10, 1)
#                  ,resNumTopPala = 30):
    
def fitLDA_Model(corpus_tfidf_ 
                 ,dictionary_ 
                 ,rangAlpha = np.arange(0.2, 0.6, 0.05)
                 ,rangEta = np.arange(0.4, 3, 0.4)
                 ,rangTopics = range(3, 6, 1)
                 ,resNumTopPala = 30):

    rangAlpha = rangAlpha.round(decimals=2)
    rangEta = rangEta.round(decimals=2)
    
    resultMatrix_ = []
    for alpha_ in rangAlpha:
        for eta_ in rangEta:
            for num_topics_ in rangTopics:
                print(alpha_, eta_, num_topics_ )
                #Ajustar el modelo con la configuración que corresponda en el for
                lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf_, 
                                                             id2word = dictionary_, 
                                                             alpha = alpha_, 
                                                             eta = eta_, 
                                                             num_topics = num_topics_, 
                                                             workers=7,
                                                             random_state=42)
                
                #Obtener la distribución de los documentos en los tópicos
                topics = lda_model_tfidf.get_document_topics(corpus_tfidf_, 
                                                             minimum_probability=None, 
                                                             minimum_phi_value=None, 
                                                             per_word_topics=False)
                
                #Guarda el top n paralabra en el resultado, según parámetro resNumTopPala
                top_Palabras = []
                for i in range(0,num_topics_):
                    top_Palabras.extend( np.append(np.full((resNumTopPala,1), i+1), lda_model_tfidf.show_topic(i, topn=resNumTopPala), axis=1 ) )
                
                resultMatrix_.append([alpha_, eta_, num_topics_, topics, top_Palabras])
                
    return resultMatrix_

## Función lineal - arriba está la función paralelizada
def fitLDA_Model(corpus_tfidf_ 
                 ,dictionary_ 
                 ,rangAlpha = np.arange(0.1, 1, 0.1)
                 ,rangEta = range(1, 5)
                 ,rangTopics = range(4, 10, 1)
                 ,resNumTopPala = 20):
    
   
    resultMatrix_ = []
    #for alpha_ in rangAlpha:
    #    for eta_ in rangEta:
    for num_topics_ in rangTopics:
        print( num_topics_ )
        #Ajustar el modelo con la configuración que corresponda en el for
        lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf_, 
                                                     id2word = dictionary_, 
                                                     num_topics = num_topics_, 
                                                     workers=-1,
                                                     random_state=42)

        #Obtener la distribución de los documentos en los tópicos
        topics = lda_model_tfidf.get_document_topics(corpus_tfidf_, 
                                                     minimum_probability=None, 
                                                     minimum_phi_value=None, 
                                                     per_word_topics=False)

        #Guarda el top n paralabra en el resultado, según parámetro resNumTopPala
        top_Palabras = []
        for i in range(0,num_topics_):
            top_Palabras.extend( np.append(np.full((resNumTopPala,1), i+1), lda_model_tfidf.show_topic(i, topn=resNumTopPala), axis=1 ) )

        resultMatrix_.append([-1, -1, num_topics_, topics, top_Palabras])
                
    return resultMatrix_ 

### Función para calcular modelos VAR y LDA

In [12]:
def fit_VARModel(docDataF_, indicadorSerie, data_series_, num_periodos_test):

    resultSeries_ = []

    #correr el ajuste del LDA mes a mes por el numero de periodos de test seleccionados
    #en cada iteración se calcula un gridsearch con los parametros del LDA
    for i in range (0, num_periodos_test):
        
        documents=[]
        #1.1 Correr función "preprocess" enviando el texto mensualizado removiendo los periodos de test 1 a la vez 
        for texto in docDataF_[:-num_periodos_test+i].text:
            documents.append(preprocess(texto))
        print("num_mes_test:", i, "num_docs", len(documents) )
        
        #1.2 Vectorizar los textos con TFIDF, la funcion retorna el diccionario y el texto vectorizado
        dictionary, corpus_tfidf = vectorizarTextos (documents)

        #1.3 Ajustar los modelos LDA con las posible combinaciones de alpha, eta y número de topicos
        # Retorna la distribución de tópicos por cada documento de acuerdo con el modelo LDA ajustado con los diferentes parámetros
        paramsTopics_ = fitLDA_Model(corpus_tfidf, dictionary)
        
        result_ = []
        #Correr un modelo VAR para cada resultado de LDA de acuerdo con el gridsearch
        for num_model in range(0,len(paramsTopics_)):
            topics = paramsTopics_[num_model][3]
            #Crear un Dataframe de topicos en los documentos como serie de tiempo
            topics_df = pd.DataFrame(gensim.matutils.corpus2csc(topics).T.toarray())
            
            #Unir los topicos con el indicador en el mismo DataFrame
            data = indicadorSerie[:-num_periodos_test+i].reset_index().iloc[:,1:].join(topics_df).join(data_series_[:-num_periodos_test+i])
            #data.to_pickle("Data.pkl") #Guardar resultado parcial
            
            #El numero 1 indica el numero de resagos
            model = VAR(data)            
            results = model.fit(1)

            #proyectar test
            proy_var = pd.DataFrame(results.forecast(data.values[-results.k_ar:], 1))
            #print("num_model:", num_model, " proy:", proy_var.values[:,0])
            
            result_.append([ proy_var.values[:,0], indicadorSerie.iloc[-num_periodos_test+i,0] ] )
            
        #print(result_)
        
        resultSeries_.append(np.append(result_, paramsTopics_, axis=1) )        
    
    return resultSeries_

# Flujo de ejecución

In [13]:
print("Inicio =", datetime.now())
otras_series = data_series.reset_index().drop(["Fecha", "TASA_POL_MON"], axis=1)
resultSeries = fit_VARModel(docDataF, data_series[["TASA_POL_MON"]],otras_series, 24)
print("Fin =", datetime.now())

# Ultima ejecución
# Inicio = 2019-09-17 23:05:17.495484
# Fin = 2019-09-18 03:20:36.631524

Inicio = 2019-09-23 15:31:41.424481


NameError: name 'data_series' is not defined

In [ ]:
## Guardar resultado
fileObject = open("Result_17Sep_lin.pkl", 'wb')
pickle.dump(resultSeries, fileObject)
fileObject.close()

# Carga y transforma resultados para reporte

In [ ]:
## Cargar resultado guardado
fileObject2 = open("Result_17Sep_lin.pkl", 'rb')
resultLoaded = pickle.load(fileObject2)
fileObject2.close()
# Leer un registro para validar
resultLoaded[1][1][5]

## Transformar y crear arrays

In [ ]:
#Nivel 1: Mes BackTest
#Nivel 2: Modelo LDA
#Nivel 3: Proyeccion, valor Original, Parámetros, tópicos y palabras
#resultLoaded[0][0][:4]
dataModel = []
modelTopicos = []
modelPalabras = []
for nivel1 in range(0,len(resultLoaded)): 
    for nivel2 in range(0,len(resultLoaded[nivel1])): 
        
        dataModel.append( np.append([nivel1],
                           np.append([nivel2], 
                           resultLoaded[nivel1][nivel2][:5], axis=0), axis=0))
        
        lonTopicos = len(resultLoaded[nivel1][nivel2][5])
        modelTopicos.extend(np.append(np.full((lonTopicos,1), nivel1),
                            np.append(np.full((lonTopicos,1), nivel2),
                            np.append(np.indices((lonTopicos,1))[0] ,
                            gensim.matutils.corpus2csc(resultLoaded[nivel1][nivel2][5]).T.toarray(), axis=1), axis=1), axis=1))
        
        lonPalabra = len(resultLoaded[nivel1][nivel2][6])
        modelPalabras.extend(np.append(np.full((lonPalabra,1), nivel1),
                             np.append(np.full((lonPalabra,1), nivel2),
                             resultLoaded[nivel1][nivel2][6], axis=1), axis=1))

## Crear dataframes y transformar formatos

In [ ]:
#import datetime
#import calendar

#Crea Dataframe modelos
dataModel_df = pd.DataFrame(dataModel, columns=['MesBack','ModeloLDA','Proyec','OrgValue','Alpha','Eta','Topics' ])
fec_end = max(data_series.index)
lenback = max(dataModel_df.MesBack)
dataModel_df.MesBack = dataModel_df.MesBack.apply(lambda x: fec_end - relativedelta(months=lenback-x))

#Crea Dataframe topicos
modelTopicos_df = pd.DataFrame(modelTopicos)
modelTopicos_df.rename(columns= {0: "MesBack", 1:"Modelo", 2:"MesDocumento", 3:'T1',4:'T2',5:'T3',6:'T4',7:'T5',8:'T6',9:'T7',10:'T8',11:'T9'}, inplace = True) 
modelTopicos_df.MesBack = modelTopicos_df.MesBack.apply(lambda x: fec_end - relativedelta(months=lenback-x))

#Crea Dataframe palabras
modelPalabras_df = pd.DataFrame(modelPalabras, columns=['MesBack','ModeloLDA','Topic','Palabra','Importancia'])
modelPalabras_df.MesBack = modelPalabras_df.MesBack.apply(lambda x: fec_end - relativedelta(months=lenback-x.astype(int)))
modelTopicos_df.Modelo = modelTopicos_df.Modelo.astype(int)
fec_ini = min(data_series.index)
modelTopicos_df.MesDocumento = modelTopicos_df.MesDocumento.apply(lambda x: fec_ini + relativedelta(months=x))


In [ ]:
dataModel_df.head()

In [ ]:
modelTopicos_df.head()

In [ ]:
modelPalabras_df.head()

## Exportar a csv

In [ ]:
dataModel_df.to_csv("Modelos.csv", index = False)
modelTopicos_df.to_csv("Topicos.csv", index = False)
modelPalabras_df.to_csv("Palabras.csv", index = False)

# Codigo de pruebas

In [ ]:
#Ejemplo para:
#Nivel 1: Primer mes de Back Test 0
#Nivel 2: Modelo LDA 0
#Nivel 3: Se imprime a continuación
mes= 3
model = 10
print("Error",resultLoaded[mes][model][0], "Alpha: ",resultLoaded[mes][model][1], " Eta:", resultLoaded[mes][model][2], " NumTopics:", resultLoaded[mes][model][3])

#Convertir el array top de palabras en un DataFrame
palabrasModel = pd.DataFrame(resultLoaded[mes][model][5], columns=['Topic','Palabra','Importancia'])
palabrasModel.head()

In [ ]:
######################################
##########       TEST      ###########
######################################

documents=[]

#1.1 Correr función "preprocess" enviand el texto de al cual se le va aplicar LDA, el resultado se guarda en el vector documents
for texto in docDataF.text:    
    documents.append(preprocess(texto))

#1.2 Vectorizar los textos con TFIDF, la funcion retorna el diccionario y el texto vectorizado
dictionary, corpus_tfidf = vectorizarTextos (documents)

num_topics_=8

lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                             id2word = dictionary, 
                                             alpha = 0.5, 
                                             eta = 1, 
                                             num_topics=num_topics_, 
                                             workers=3, 
                                             random_state=42)
                
#Obtener la distribución de los documentos en los tópicos
topics = lda_model_tfidf.get_document_topics(corpus_tfidf, 
                                             minimum_probability=None, 
                                             minimum_phi_value=None, 
                                             per_word_topics=False)



In [ ]:
# Prueba Paralelo
otras_series = data_series.reset_index().drop(["Fecha", "TASA_POL_MON"], axis=1)
resultSeries = fit_VARModel(docDataF, data_series[["TASA_POL_MON"]],otras_series, 2)

### Función para probar todos los inidcadores 
topicsResult = []
for column in data_series.columns.tolist():
    print(column)
    otras_series = data_series.reset_index().drop(["Fecha", column], axis=1)
    resultSeries = fit_VARModel(docDataF, data_series[[column]],otras_series, 36)
    topicsResult.append(resultSeries)

In [ ]:
print(gensim.matutils.corpus2csc(topics).T.toarray())

In [ ]:
topics_df = pd.DataFrame(gensim.matutils.corpus2csc(topics).toarray())
topics_df